In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, precision_recall_curve
import seaborn as sns

from pickle import dump, load

---
Load in the trainval- and test-data

In [2]:
data_trainval = pd.read_csv(os.path.join('data', 'train.csv'))
X_trainval = data_trainval[data_trainval.columns.drop('Survived')]
y_trainval = data_trainval.Survived

X_test = pd.read_csv(os.path.join('data', 'test.csv'))

y_example = pd.read_csv(os.path.join('data', 'gender_submission.csv'))
y_example[:5]

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


---
Build the preprocessing pipeline by running the script (see previous notebooks for details).

In [3]:
%run build_preprocess_pipe.py

---
Load in the classifier. Fit to all the known data.

In [4]:
with open("vote_clf.pkl", "rb") as file:
    vote_clf = load(file)

In [5]:
vote_clf.fit(X_trainval, y_trainval)

VotingClassifier(estimators=[('K Nearest Neighbors', Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('transform', FeatureUnion(n_jobs=None,
       transformer_list=[('numerical_transformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0,
         transformer_..., fit_prior=True, norm=False),
          learning_rate=0.8, n_estimators=100, random_state=123))]))],
         flatten_transform=None, n_jobs=None, voting='soft',
         weights=[1, 1, 1, 1])

In [9]:
vote_clf.score(X_trainval, y_trainval)

0.8630751964085297

---
Predict on *X_test*. Append to the *PassengerId*.

In [14]:
my_submission = pd.DataFrame({"PassengerId": X_test.PassengerId,
                              "Survived": vote_clf.predict(X_test)})

In [15]:
my_submission[:5]

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [16]:
my_submission.to_csv("my_submission.csv", index=None)